In [ ]:
import os
import re
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage
from langchain_community.vectorstores import Qdrant
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import ChatPromptTemplate
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Jika dijalankan di notebook:
try:
    from IPython.display import display, Markdown
    IPYTHON_MODE = True
except ImportError:
    IPYTHON_MODE = False

# === Konfigurasi ===
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
QDRANT_COLLECTION_NAME = "rag-pembelajaran"
TEXT_FOLDER = "./docs"

# === Fungsi Preprocessing Dokumen ===
def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s.,]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# === Step 1: Load dan Preprocess Dokumen ===
loader = DirectoryLoader(TEXT_FOLDER, glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()

for doc in documents:
    doc.page_content = preprocess_text(doc.page_content)

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs_split = splitter.split_documents(documents)

# === Step 2: Embedding & Vector Store ===
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

qdrant_client = QdrantClient(url="http://localhost:6333")

qdrant_client.recreate_collection(
    collection_name=QDRANT_COLLECTION_NAME,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

qdrant = Qdrant.from_documents(
    documents=docs_split,
    embedding=embedding_model,
    url="http://localhost:6333",
    prefer_grpc=False,
    collection_name=QDRANT_COLLECTION_NAME,
    force_recreate=True
)

# === Step 3: Retriever + Contextual Compression ===
base_retriever = qdrant.as_retriever(search_kwargs={"k": 5})

llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="gemma2-9b-it",
    temperature=0.2,
)

compressor = LLMChainExtractor.from_llm(llm)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever,
)

# === Step 4: Reasoning Prompt ===
system_prompt = SystemMessage(
    content="""
Anda adalah asisten reasoning cerdas.
Tugas Anda:
- Gabungkan informasi dari berbagai sumber dokumen.
- Lakukan penalaran logis seperti membandingkan, mengklasifikasi, menyimpulkan, dan memberi rekomendasi.
- Susun jawaban sistematis: Pendahuluan, Analisis, Kesimpulan.
Jika jawaban tidak ditemukan, jujurlah.

Gunakan bahasa formal dan mudah dipahami.
"""
)

prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    ("human", "Dokumen terkait:\n{context}\n\nPertanyaan:\n{question}")
])

# === Step 5: RAG Chain ===
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
        "document_variable_name": "context"
    }
)

# === Step 6: Jalankan CLI ===
if __name__ == "__main__":
    print("🧠 Sistem RAG Reasoning Siap Digunakan.")
    while True:
        query = input("\nMasukkan pertanyaan Anda (atau 'exit'): ")
        if query.strip().lower() == "exit":
            print("👋 Keluar dari sistem.")
            break

        result = rag_chain.invoke(query)
        jawaban = result["result"]
        sumber = result["source_documents"]

        # Buat konten markdown
        md_content = f"""
## 🤖 **Tutor:**

{jawaban}

---

"""
        display(Markdown(md_content))


/tmp/ipykernel_8839/3924759934.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/taqin/code/ai/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_8839/3924759934.py:52: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instea

🧠 Sistem RAG Reasoning Siap Digunakan.



## 🤖 **Tutor:**

## Cara Belajar yang Bagus dan Efisien

**Pendahuluan:**

Efisiensi dalam pembelajaran merupakan kunci keberhasilan dalam menyerap pengetahuan dan keterampilan.  Metode belajar yang efektif tidak hanya berfokus pada penyerapan informasi, tetapi juga pada pemahaman, penerapan, dan retensi pengetahuan. 

**Analisis:**

Berdasarkan dokumen yang diberikan, terdapat beberapa karakteristik belajar yang baik dan efisien:

1. **Pembelajaran Aktif:**  Metode ini mendorong siswa untuk terlibat aktif dalam proses belajar, bukan hanya sebagai penerima pasif informasi.  Contohnya, melalui wawancara, simulasi, analisis dokumen, dan proyek yang bermakna. 
2. **Pembelajaran Kontekstual:**  Penyerapan pengetahuan lebih efektif ketika dikaitkan dengan konteks nyata dan relevan bagi siswa.  Misalnya, belajar matematika melalui proyek anggaran sekolah, bukan hanya rumus abstrak.
3. **Interaksi Sosial:**  Belajar bersama teman sekelas melalui diskusi, kerja kelompok, dan berbagi perspektif dapat memperkaya pemahaman dan meningkatkan retensi informasi.
4. **Pembelajaran yang Terstruktur:**  Struktur yang jelas dalam pembelajaran, seperti kurikulum yang terdefinisi, tujuan pembelajaran yang spesifik, dan metode pengajaran yang sistematis, membantu siswa memahami ekspektasi dan konsekuensi, serta memudahkan evaluasi dan replikasi pembelajaran.
5. **Pembelajaran yang Terukur dan Objektif:**  Metode pembelajaran yang memungkinkan pengukuran dan penilaian hasil belajar secara konkret dan objektif, seperti tes, proyek, dan portofolio, membantu guru dan siswa memantau kemajuan belajar dan mengidentifikasi area yang perlu ditingkatkan.

**Kesimpulan:**

Cara belajar yang bagus dan efisien menggabungkan pembelajaran aktif, kontekstual, sosial, terstruktur, dan terukur.  Metode ini mendorong keterlibatan siswa, pemahaman mendalam, retensi pengetahuan, dan perkembangan keterampilan yang berkelanjutan.




---




## 🤖 **Tutor:**

## Visualisasi Pembelajaran Aktif

**Pendahuluan:**

Dokumen yang diberikan membahas tentang pembelajaran aktif, menekankan pentingnya keterlibatan siswa dalam proses belajar.  Visualisasi dapat menjadi alat yang efektif untuk memperjelas konsep pembelajaran aktif dan bagaimana penerapannya dalam praktik.

**Analisis:**

Berdasarkan dokumen, berikut beberapa elemen pembelajaran aktif yang dapat divisualisasikan:

* **Pergeseran dari Penerimaan Informasi ke Penemuan:** 
    * **Visual:** Sebuah diagram yang menunjukkan siswa dari posisi pasif (mendengarkan) berubah menjadi posisi aktif (menanyakan, mencoba, menemukan) dalam proses belajar.
* **Pembelajaran Kontekstual:**
    * **Visual:** Sebuah peta konsep yang menghubungkan konsep matematika dengan proyek anggaran sekolah, menunjukkan bagaimana pembelajaran terjadi dalam konteks nyata.
* **Interaksi Sosial:**
    * **Visual:** Sebuah diagram jaringan yang menggambarkan siswa berdiskusi, bekerja dalam kelompok, dan berbagi perspektif, menekankan pentingnya kolaborasi dalam pembelajaran aktif.
* **Peningkatan Keterampilan Berpikir Tingkat Tinggi:**
    * **Visual:** Sebuah flowchart yang menunjukkan bagaimana pembelajaran aktif melalui simulasi sidang PPKI atau analisis dokumen bersejarah dapat membantu siswa mengembangkan keterampilan berpikir kritis, analitis, dan kreatif.

**Kesimpulan:**

Visualisasi dapat menjadi alat yang efektif untuk memahami dan mempromosikan pembelajaran aktif. Dengan menggunakan diagram, peta konsep, dan flowchart, kita dapat menggambarkan pergeseran peran siswa, pembelajaran kontekstual, interaksi sosial, dan pengembangan keterampilan berpikir tingkat tinggi yang menjadi ciri khas pembelajaran aktif. 




---




## 🤖 **Tutor:**

## Flowchart: Belajar Efektif di Era AI dan Teknologi

**Pendahuluan:**

Era AI dan teknologi menghadirkan peluang baru dalam pembelajaran.  Flowchart ini merangkum bagaimana menerapkan prinsip-prinsip pembelajaran aktif, kontekstual, dan sosial untuk belajar efektif di era digital ini.

**Analisis:**

1. **Identifikasi Tujuan Pembelajaran:**
    - Tentukan apa yang ingin Anda capai melalui pembelajaran. 
    - Gunakan AI untuk membantu merumuskan tujuan pembelajaran yang spesifik, terukur, dapat dicapai, relevan, dan berbatas waktu (SMART).

2. **Pilih Sumber Belajar yang Tepat:**
    - Manfaatkan platform pembelajaran online, aplikasi edukasi, dan sumber daya AI seperti chatbot edukatif.
    - Pilih sumber yang relevan dengan tujuan pembelajaran dan gaya belajar Anda.

3. **Terapkan Pembelajaran Aktif:**
    - Jangan hanya membaca atau menonton, aktiflah dalam proses belajar.
    - Gunakan AI untuk membuat kuis, simulasi, dan permainan edukatif yang mendorong Anda untuk berpikir kritis dan memecahkan masalah.

4. **Belajar dalam Konteks Nyata:**
    - Hubungkan materi pembelajaran dengan dunia nyata melalui proyek, studi kasus, dan aplikasi praktis.
    - Gunakan AI untuk mencari contoh kasus dan menghubungkan konsep dengan situasi nyata.

5. **Berinteraksi Sosial:**
    - Bergabunglah dengan forum online, grup belajar, dan diskusi dengan teman atau mentor.
    - Manfaatkan AI untuk terhubung dengan ahli di bidang yang Anda minati.

6. **Evaluasi dan Refleksi:**
    - Gunakan AI untuk melacak kemajuan Anda dan mengidentifikasi area yang perlu ditingkatkan.
    - Refleksikan proses belajar Anda dan sesuaikan strategi belajar Anda untuk lebih efektif.

**Kesimpulan:**

Belajar efektif di era AI dan teknologi membutuhkan strategi yang terencana dan aktif. Dengan menggabungkan prinsip pembelajaran aktif, kontekstual, dan sosial, serta memanfaatkan potensi AI, Anda dapat memaksimalkan proses belajar dan mencapai tujuan pembelajaran Anda.




---




## 🤖 **Tutor:**

## Best Practice Gaya Belajar untuk Mahasiswa Sistem Informasi

**Pendahuluan:**

Sebagai mahasiswa Sistem Informasi, Anda beruntung berada di era dimana teknologi pembelajaran semakin canggih.  Pilihan gaya belajar terbaik untuk Anda akan bergantung pada preferensi pribadi, gaya belajar, dan kebutuhan spesifik program studi. 

**Analisis:**

Dokumen yang Anda berikan menyoroti beberapa tren teknologi dalam pembelajaran, seperti:

* **Learning Management System (LMS):**  Platform ini menyediakan akses ke materi kuliah, tugas, forum diskusi, dan alat penilaian secara terpusat. LMS dapat membantu Anda mengatur jadwal belajar, melacak kemajuan, dan berinteraksi dengan dosen dan teman sekelas.
* **Video Interaktif:**  Video pembelajaran interaktif dapat membantu Anda memahami konsep-konsep kompleks dengan cara yang lebih menarik dan mudah diingat. Video ini seringkali dilengkapi dengan kuis, latihan, dan simulasi yang dapat menguji pemahaman Anda.
* **AI-Based Tutoring:**  Sistem pembelajaran berbasis AI dapat memberikan bimbingan dan dukungan individual berdasarkan kebutuhan belajar Anda. AI tutor dapat menjawab pertanyaan, memberikan umpan balik pada tugas, dan mengidentifikasi area yang perlu diperkuat.

**Kesimpulan:**

Berikut adalah beberapa best practice gaya belajar yang dapat Anda pertimbangkan sebagai mahasiswa Sistem Informasi:

1. **Manfaatkan LMS secara maksimal:**  Gunakan LMS untuk mengakses materi kuliah, mengerjakan tugas, dan berpartisipasi dalam forum diskusi. 
2. **Kombinasikan metode belajar:**  Jangan hanya bergantung pada satu metode belajar. Gabungkan pembelajaran pasif (membaca, menonton video) dengan pembelajaran aktif ( mengerjakan latihan, berdiskusi).
3. **Cari mentor atau kelompok belajar:**  Bergabung dengan kelompok belajar atau mencari mentor dapat membantu Anda memahami materi dengan lebih baik dan membangun jaringan profesional.
4. **Kembangkan literasi digital:**  Tingkatkan kemampuan Anda dalam menggunakan teknologi untuk belajar. Pelajari cara mencari informasi secara efektif, menggunakan alat pembelajaran online, dan berinteraksi dalam lingkungan digital.
5. **Evaluasi dan sesuaikan gaya belajar Anda:**  Perhatikan metode belajar yang paling efektif untuk Anda dan sesuaikan gaya belajar Anda sesuai kebutuhan.


Penting untuk diingat bahwa tidak ada satu gaya belajar yang cocok untuk semua orang.  Cobalah berbagai metode dan temukan kombinasi yang paling efektif untuk Anda. 


---

